## Marquez - a reference backend service

In [1]:
import json,requests
marquez_url = "http://api:5000" ## this may depend on your local setup
if (requests.get("{}/api/v1/namespaces".format(marquez_url)).status_code == 200):
    print("Marquez is OK.")
else:
    print("Cannot connect to Marquez")

Marquez is OK.


Marquez URL is http://localhost:3000

In [6]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.master('local')
         .appName('Food Delivery')
         .config('spark.extraListeners', 'io.openlineage.spark.agent.OpenLineageSparkListener')
         .config('spark.jars', '/home/jovyan/jars/openlineage-spark-0.27.2.jar,/home/jovyan/jars/postgresql-42.6.0.jar')
         .config('spark.openlineage.transport.type', 'http')
         .config('spark.openlineage.transport.url', 'http://api:5000')
         .config('spark.openlineage.facets.disabled', '[spark_unknown;spark.logicalPlan]')
         .config('spark.openlineage.namespace', 'food-delivery')
         .enableHiveSupport()
         .getOrCreate())

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Exception when registering SparkListener
	at org.apache.spark.SparkContext.setupAndStartListenerBus(SparkContext.scala:2563)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:643)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngi

## Most popular restaurants in Berlin last week

In [ ]:
df = spark.sql('''
  SELECT r.id, r.name, r.address, count(order_id) as orders
  FROM public.restaurants r
  JOIN public.orders_7_days o ON (o.restaurant_id = r.id) 
  GROUP BY r.id, r.name, r.address
''')

### Write dataset to a table

In [ ]:
df.write.mode("overwrite").saveAsTable('public.top_berlin_restarurants')

### Write the same dataset to postgres table 

In [ ]:
df \
    .write \
    .mode("overwrite") \
    .jdbc("jdbc:postgresql://db/data", "top_berlin_restarurants", 
          properties={
              "user": "data", 
              "password": "data", 
              "schema": "data", 
              "driver": "org.postgresql.Driver"
          }
         )

Marquez:
 * Job in [Marquez UI](http://localhost:3000/lineage/job/jelly-world/jelly_data_mesh.execute_create_data_source_table_as_select_command).
 * [Lineage in Marquez API](http://localhost:3000/api/v1/column-lineage?nodeId=dataset:file:%2Fhome%2Fjovyan%2Fnotebooks%2Fspark-warehouse%2Fjelly_daily_green)